# Análisis y preprocesamiento de los datos

Se explorarán, en pequeños experimentos, distintas formas de representación de los datos del corpus WiNER (Ghaddar y Langlais 2017) para utilizarlos en la tarea de reconocimiento de entidades nombradas. Para esto se exploran distintas combinaciones de vectores de palabras como representación de una instancia de entrenamiento (Iacobacci et al. 2016).

## Descripción del Corpus WiNER

* Documents.tar.bz2 : este archivo contiene 3239540 artículos de Wikipedia repartidos en 3223 archivos. Cada archivo contiene aproximadamente 1000 artículos nombrados por sus respectivos IDs. Los artículos están indexados por su wikiID seguidos de oraciones (una por línea), donde las palabras son remplazadas por sus ids.

      ID <number>
      1234 4522 23 4 4567
      456 21 9890 123 7 0

* document.vocab : este archivo contiene el mapeo de palabras (case sensitive); el formato es: 
 
      palabra #ocurrencias
    
  El ID de cada palabra es el número de línea en la cual ocurre.

In [1]:
import pandas as pd
import numpy as np
import time
import gensim
import os
import itertools
from gensim.models import Word2Vec, KeyedVectors
from math import floor

from corpus import create_coarseNE_df
from corpus import spread_artID
from embeddings import fill_w2vMatrix, exponential_decay

### Cargamos los datos

In [2]:
word_mapping = pd.read_csv('./corpus_WiNER/document.vocab', sep=' ', header=None, 
                           names=['word', 'frequency'], keep_default_na=False) 
                        # con esto evito que el parser trate como nan value a algunas palabras.

/users/ekokic/thesis_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(word_mapping.shape)
word_mapping.head()

(2021984, 2)


,word,frequency
0,the,73411953
1,",",68044881
2,.,54241850
3,of,40550466
4,and,34602894


In [4]:
len(word_mapping['word'].unique())

2021983

### Cargamos los artículos del Documento "0"

In [5]:
doc_0 = pd.read_csv('./corpus_WiNER/Documents/0', sep='ID', engine='python', header=None, names=['sentence', 'art_ID'])

In [6]:
doc_0.shape

(130209, 2)

In [7]:
doc_0.head()

,sentence,art_ID
0,NaN,431388.0
1,650 5590 753942 12 189 243 1 2039 47 257 682 1...,NaN
2,34 23207 1523 4 57193 15 10777 14353 4 157019 ...,NaN
3,418 0 1858 101 0 1322 1 5590 18 860 729 14 92 ...,NaN
4,650 5590 1523 8 1136 15 11301 575 1156 1 2746 ...,NaN


Vamos a asociarle a cada oración el ID del artículo al cual pertenece.

Es importante recordar que el orden de las oraciones está dado por los índices del dataframe.

In [8]:
art_ID_list = doc_0['art_ID'].tolist()
art_ID = 0
for idx, elem in enumerate(art_ID_list):
    if not np.isnan(elem):
        art_ID = elem
    else:
        art_ID_list[idx] = art_ID
doc_0['art_ID'] = art_ID_list

Removemos ahora las filas con NaN que contenian los ID de los artículos inicialmente.

In [9]:
doc_0 = doc_0.dropna()

In [10]:
print('El documento contiene {} oraciones.'.format(doc_0.shape[0]))
print('El documento contiene {} artículos'.format(len(doc_0['art_ID'].unique())))

El documento contiene 128395 oraciones.
El documento contiene 1814 artículos


Hacemos que cada sentencia sea una lista de palabras codificadas y casteamos a int

In [11]:
doc_0['sentence'] = doc_0['sentence'].map(lambda x: list(map(int, x.split(' '))))

In [12]:
doc_0.head()

,sentence,art_ID
1,"[650, 5590, 753942, 12, 189, 243, 1, 2039, 47,...",431388.0
2,"[34, 23207, 1523, 4, 57193, 15, 10777, 14353, ...",431388.0
3,"[418, 0, 1858, 101, 0, 1322, 1, 5590, 18, 860,...",431388.0
4,"[650, 5590, 1523, 8, 1136, 15, 11301, 575, 115...",431388.0
5,"[152, 642, 16, 8143, 23122, 20, 0, 662955, 16,...",431388.0


Nos quedamos con un artículo

In [13]:
article = doc_0[doc_0.art_ID == 1000]
article.head()

,sentence,art_ID
2756,"[63903, 24730, 9, 7, 3035, 4104, 7584, 1, 355,...",1000.0
2757,"[3654, 16, 2502, 39413, 1, 24730, 9, 44, 3, 99...",1000.0
2758,"[24730, 35, 49, 3521, 15, 575, 1, 15, 2440, 1,...",1000.0
2759,"[152, 112, 8, 2037, 20, 52, 54, 3035, 17572, 3...",1000.0
2760,"[64, 62, 5787, 1132, 15, 0, 144, 24730, 1312, ...",1000.0


Reconstruimos la primera oración del artículo

In [14]:
def sentence_decoder(sentence, word_mapping):
    dec_sentence = []
    for idx in sentence:
        mapped_w = word_mapping.loc[idx, 'word']
        dec_sentence.append(mapped_w)
    return dec_sentence 

In [15]:
# Escribimos article.columns.get_loc('sentence') para evitar hardcodear el índice correspondiente
# a la columna 'sentence' que en este caso es 0
sentence = article.iloc[0, article.columns.get_loc('sentence')]

In [16]:
dec_sentence = sentence_decoder(sentence, word_mapping)
' '.join(dec_sentence)

'Hercule Poirot is a fictional Belgian detective , created by Agatha Christie .'

### Word embeddings utilizando el modelo pre-entrenado word2vec de Google

Utilizaremos la librería Gensim https://radimrehurek.com/gensim/

Cargamos Google's pre-trained Word2Vec model.

Utilizando KeyedVectors para cargar el modelo tiene la desventaja de que no se puede seguir entrenando. Pero es más eficiente que utilizar gensim.models.Word2Vec
https://radimrehurek.com/gensim/models/keyedvectors.html#module-gensim.models.keyedvectors

In [17]:
start = time.time()
# model = KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)
# model.save('./models/word2vecGoogle.model')
w2v_model = KeyedVectors.load('./models/word2vecGoogle.model')
end = time.time()
print('demora: {}'.format(end-start))
# model = Word2Vec.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)

demora: 8.36343264579773


In [18]:
print('Cantidad de word embeddings: {}'.format(len(w2v_model.vectors)))

Cantidad de word embeddings: 3000000


In [19]:
print('Dimensionalidad de los vectores: {}'.format(w2v_model.vector_size))

Dimensionalidad de los vectores: 300


## Exploremos distintas combinaciones de vectores de palabras

### Concatenación

Este método consiste en concatenar los vectores de palabras que rodean una palabra objetivo en un vector más grande, que tiene un tamaño igual a las dimensiones agregadas de todos las proyecciones (embeddings) individuales.

- $w_{ij}$ = peso asociado con la i-ésima dimensión del vector de la j-ésima palabra en la oración. NOTA: con los vectores de palabras de una oración se forma una matriz $w^{\space D\space x\space L}$ donde $L$ es la cantidad de palabras de esa oración.
- $D$ = dimensionalidad de los word vectors originales. Por ejemplo, al usar el modelo word2vec de Google se tiene $D$ = 300.
- $W$ = tamaño de ventana que se define como el número de palabras en un solo lado.

Nos interesa representar el contexto de la I-ésima palabra de la oración. 

La i-ésima dimensión del vector de concatenación, que tiene un tamaño de $2 W D$, se calcula de la siguiente manera:

$$ e_{i} =\begin{cases} 
      w_{i\space mod \space D,\space\space I \space - \space W \space + \space \left\lfloor{\frac{i}{D}}\right\rfloor} & \left\lfloor{\frac{i}{D}}\right\rfloor < W \\
      w_{i\space mod \space D,\space\space I \space - \space W \space + \space 1\space  +\space\left\lfloor{\frac{i}{D}}\right\rfloor} & c.c.
   \end{cases}$$
   

<br>
Al tomar una ventana simétrica, se realiza un relleno (padding) con ceros a izquierda y/o derecha según corresponda para mantener la misma dimesionalidad en cada nuevo vector generado.

### Promedio

Como su nombre indica, se calcula el centroide de los embeddings de todas las palabras circundantes. La fórmula divide cada dimensión en $2W$ ya que el número de palabras del contexto es dos veces el tamaño de la ventana:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} \frac{w_{ij}}{2W}$$

### Decaimiento fraccional

Una tercera estrategia para construir un vector de carácteristicas en base a los embeddings de palabras contextuales está inspirada en la forma en que Word2vec combina las palabras en el contexto. Aquí, se supone que la importancia de una palabra para nuestra representación es inversamente proporcional a su distancia respecto a la palabra objetivo.

Por lo tanto, las palabras contextuales se ponderan en función de su distancia de la palabra objetivo:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} w_{ij} *\frac{W - \lvert I-j\rvert}{W}$$


### Decaimiento exponencial

Funciona de manera similar al decaimiento fraccional, que le da más importancia al contexto cercano, pero en este caso la ponderación se realiza exponencialmente:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} w_{ij} * (1 - \alpha)^{\lvert \space I\space-\space j\space\rvert\space-\space1}$$

donde $\alpha = 1 - 0.1^{(W-1)^{-1}}$ es el parámetro de decaimiento. Elegimos el parámetro de tal manera que las palabras inmediatas que rodean a la palabra objetivo contribuyen 10 veces más que las últimas palabras en ambos lados de la ventana.

## Exploremos CoarseNE.tar.bz2

Contiene menciones anotadas automáticamente con etiquetas de entidades nombradas (PER, LOC, ORG y MISC).

El formato es:

    ID artID
    sentIdx begin end entityType
    
donde entityType[0] = PER | entityType[1] = LOC | entityType[2] = ORG | entityType[3] = MISC

In [20]:
coarseNE_0 = pd.read_csv('./corpus_WiNER/CoarseNE/0', sep='ID', engine='python', header=None, names=['named-entity', 'art_ID'])

In [21]:
print(coarseNE_0.shape)
coarseNE_0.head()

(259470, 2)


,named-entity,art_ID
0,NaN,1000.0
1,0\t0\t2\t0,NaN
2,0\t5\t6\t1,NaN
3,0\t10\t12\t0,NaN
4,1\t2\t4\t0,NaN


Aplicamos el mismo truco que utilizamos en los documentos para propagar los art_ID según corresponda

In [22]:
coarseNE_0 = spread_artID(coarseNE_0)
coarseNE_0 = coarseNE_0.dropna()

In [23]:
print('coarseNE_0 contiene {} entidades.'.format(coarseNE_0.shape[0]))
print('coarseNE_0 contiene {} artículos'.format(len(coarseNE_0['art_ID'].unique())))

coarseNE_0 contiene 257656 entidades.
coarseNE_0 contiene 1810 artículos


Creamos nuevas columnas con la info de la columna named-entity para mejor manipulación

In [24]:
def get_tag(x):
    tags = ['PER', 'LOC', 'ORG', 'MISC']
    return tags[x]

In [25]:
coarseNE_0['named-entity'] = coarseNE_0['named-entity'].map(lambda x: x.split('\t'))
coarseNE_0['senIdx'] = coarseNE_0['named-entity'].map(lambda x: int(x[0]))
coarseNE_0['begin'] = coarseNE_0['named-entity'].map(lambda x: int(x[1]))
coarseNE_0['end'] = coarseNE_0['named-entity'].map(lambda x: int(x[2]))
coarseNE_0['entityType'] = coarseNE_0['named-entity'].map(lambda x: get_tag(int(x[3])))
coarseNE_0 = coarseNE_0.drop(columns='named-entity')

In [26]:
coarseNE_0.head()

,art_ID,senIdx,begin,end,entityType
1,1000.0,0,0,2,PER
2,1000.0,0,5,6,LOC
3,1000.0,0,10,12,PER
4,1000.0,1,2,4,PER
5,1000.0,1,5,6,PER


No todos los artículos que ocurren en Documents/0 se encuentran en CoarseNE/0

Notar que esto sucede porque esos artículos no contienen entidades nombradas.

In [27]:
print('Artículos que están presentes en Documents/0 pero no en CoarseNE/0: {}'
      .format(list(set(doc_0.art_ID.unique()) - set(coarseNE_0.art_ID.unique()))))

Artículos que están presentes en Documents/0 pero no en CoarseNE/0: [431177.0, 432375.0, 432318.0, 10263.0]


In [28]:
article = doc_0[doc_0.art_ID == 431177]
for sentence in article.sentence.values:
    dec_sentence = sentence_decoder(sentence, word_mapping)
    print(' '.join(dec_sentence))

In economics , economics of location is a strategy used by firms in a monopolistic competition environment .
Unlike a product differentiation strategy , where firms make their products different in order to attract customers , the economics of location strategy causes firms to produce similar or identical products .


### Veamos como están anotadas las entidades nombradas de una oración en particular 

In [29]:
article = doc_0[doc_0.art_ID == 1000]
sentence = article.iloc[0, article.columns.get_loc('sentence')]
dec_sentence = sentence_decoder(sentence, word_mapping)
' '.join(dec_sentence)

'Hercule Poirot is a fictional Belgian detective , created by Agatha Christie .'

In [30]:
art_entities = coarseNE_0[coarseNE_0.art_ID == 1000]
entities_sen_0 = art_entities[art_entities.senIdx == 0]

In [31]:
for idx, row in entities_sen_0.iterrows():
    print('{} : {}'.format(' '.join(dec_sentence[row['begin']:row['end']]), row['entityType']))

Hercule Poirot : PER
Belgian : LOC
Agatha Christie : PER


In [32]:
doc77_df = pd.read_pickle('./corpus_WiNER/docs_df/doc_77')
article_df = doc77_df[doc77_df.art_ID == 145492]
dec_sentence = article_df.iloc[0, article_df.columns.get_loc('sentence')]

coarseNE_77 = pd.read_pickle('./corpus_WiNER/coarseNE_df/coarseNE_77')
art_entities = coarseNE_77[coarseNE_77.art_ID == 145492]
sen_entities_0 = art_entities[art_entities.senIdx == 0]

In [33]:
def entityListFromSentence(sen_entities_df, sen_length):
    entities = []
    i = 0
    for _, row in sen_entities_df.iterrows():
        while i < row['begin']:
            entities.append('O')
            i += 1
        while i < row['end']:
            entities.append(row['entityType'])
            i += 1
    while i < sen_length:
        entities.append('O')
        i += 1
    return entities

In [34]:
# # TODO: en este metodo se podria ver de hacer el dropeo
# # de algunas palabras y/o oraciones que no son/no contienen entidades

# def entityListFromArticle(article_df, art_entities_df):
#     art_entities_list = []
#     # drop=True to avoid the old index being added as a column
#     article_df = article_df.reset_index(drop=True)
#     article_df['sen_length'] = article_df['sentence'].map(lambda x: len(x))
#     for idx, row in article_df.iterrows():
#         sen_entities_list = []
#         # We take the df with the entities of each sentence
#         sen_entities_df = art_entities_df[art_entities_df.senIdx == idx]
#         sen_length = row['sen_length']
#         # An empty dataframe means that the sentence doesn't have any entity
#         if sen_entities_df.empty:
#             sen_entities_list = ['O' for _ in range(sen_length)]
#         else:
#             sen_entities_list = entityListFromSentence(sen_entities_df, sen_length)   
#         art_entities_list += sen_entities_list
        
#     return art_entities_list

In [35]:
print(article_df.head())
print(art_entities.head())

                                            sentence    art_ID
1  [Belton, House, is, a, Grade, I, listed, count...  145492.0
2  [The, mansion, is, surrounded, by, formal, gar...  145492.0
3  [Belton, has, been, described, as, a, compilat...  145492.0
4  [The, house, has, also, been, described, as, t...  145492.0
5  [Only, Brympton, d'Evercy, has, been, similarl...  145492.0
         art_ID  senIdx  begin  end entityType
42515  145492.0       0      0    2        LOC
42516  145492.0       0     10   11        LOC
42517  145492.0       0     12   13        LOC
42518  145492.0       0     14   15        LOC
42519  145492.0       0     16   17        LOC


In [36]:
# TODO: en este metodo se podria ver de hacer el dropeo
# de algunas palabras y/o oraciones que no son/no contienen entidades

def entityListFromArticle(article_df, art_entities_df):
    art_entities_list = []
    # drop=True to avoid the old index being added as a column
    article_df = article_df.reset_index(drop=True)
    article_df['sen_length'] = article_df['sentence'].map(lambda x: len(x))    
    
    for idx, row in article_df.iterrows():
        sen_entities_list = []
        # We take the df with the entities of each sentence
        sen_entities_df = art_entities_df[art_entities_df.senIdx == idx]
        sen_length = row['sen_length']
        # An empty dataframe means that the sentence doesn't have any entity
        if sen_entities_df.empty:
            sen_entities_list = ['O' for _ in range(sen_length)]
        else:
            sen_entities_list = entityListFromSentence(sen_entities_df, sen_length)
            
#         article_df['entity_list'] = sen_entities_list
        
        
        art_entities_list += sen_entities_list
        
    return art_entities_list

In [37]:
article_df.head()

,sentence,art_ID
1,"[Belton, House, is, a, Grade, I, listed, count...",145492.0
2,"[The, mansion, is, surrounded, by, formal, gar...",145492.0
3,"[Belton, has, been, described, as, a, compilat...",145492.0
4,"[The, house, has, also, been, described, as, t...",145492.0
5,"[Only, Brympton, d'Evercy, has, been, similarl...",145492.0


In [38]:
for idx, row in sen_entities_0.iterrows():
    print('{} : {}'.format(' '.join(dec_sentence[row['begin']:row['end']]), row['entityType']))

Belton House : LOC
Belton : LOC
Grantham : LOC
Lincolnshire : LOC
England : LOC


In [39]:
entity_list = entityListFromArticle(article_df, art_entities)
print(' '.join(article_df['sentence'][1]))
print(entity_list[0:18])

Belton House is a Grade I listed country house in Belton near Grantham , Lincolnshire , England .
['LOC', 'LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'LOC', 'O', 'LOC', 'O', 'LOC', 'O', 'LOC', 'O']


In [40]:
article_df = article_df.reset_index(drop=True)
article_df['sen_length'] = article_df['sentence'].map(lambda x: len(x))
article_df.sen_length.sum() == len(entity_list)

True

In [41]:
print('Cantidad de tokens: {}'.format(len(entity_list)))
print('Cantidad de entidades nombradas: {}'.format(len(entity_list) - entity_list.count('O')))
print('Porcentaje de entidades nombradas: {:.2f}%'.format(399/4395*100))

Cantidad de tokens: 4395
Cantidad de entidades nombradas: 399
Porcentaje de entidades nombradas: 9.08%


En general, cada artículo contiene una proporción reducida de palabras que son entidades.

Una alternativa para evitar cómputo y uso de memoria podría ser eliminar una proporción de vectores que no representan ninguna entidad (etiquetados con 'O') a la hora de construir la matriz de palabras.

In [42]:
def w2v_strategy(strategy, sentence, D, W, w2v_model):
    return {
#         'concat': concat_vectors(sentence, D, W, w2v_model), 
#         'mean': mean_vectors(sentence, D, W, w2v_model), 
#         'frac_decay': fractional_decay(sentence, D, W, w2v_model),
        'exp_decay': exponential_decay(sentence, D, W, w2v_model),
    }[strategy]

In [43]:
def wordVectorsFromArticle(article_df, strategy, D, W, w2v_model):
    count = 0
    no_vocab_list = []
#     if strategy == 'concat':
#         word_vectors = np.empty((0, 2*W*D))
#     else:    
#         word_vectors = np.empty((0, D))
    word_vectors = []

    for _, row in article_df.iterrows():
        sen_vectors, c, l = w2v_strategy(strategy, row['sentence'], D, W, w2v_model)
        count += c
        no_vocab_list.append(l)
#         word_vectors = np.append(word_vectors, sen_vectors, axis=0)
        word_vectors.append(sen_vectors)

    # Fastest way to flatten list of arrays
    word_vectors = list(itertools.chain(*word_vectors))
    
    return word_vectors, count, no_vocab_list

In [44]:
w, count, no_vocab_list = wordVectorsFromArticle(article_df, strategy='exp_decay', D=300, 
                                                 W=5, w2v_model=w2v_model)
print('#palabras del artículo: {}'.format(len(w)))
print('#palabras que no están en el vocabulario de w2v_model: {}'.format(count))

#palabras del artículo: 4395
#palabras que no están en el vocabulario de w2v_model: 0


In [45]:
no_vocab_list[0:10]

[[], [], [], [], [], [], [], [], [], []]

Muchas palabras del artículo no están en el vocabulario del modelo word2vec de Google.

En general se trata de stopwords, signos de puntuación, números, palabras mal escritas, etc.

Frente a estos casos se opta por realizar lo siguiente: si la palabra no está en el vocabulario se la transforma a un vector de 0's.

In [46]:
# print(w.shape)
# w_df = pd.DataFrame(w)
# w_df['entityType'] = entity_list
# w_e = w_df.values
# print(w_e.shape)
# first_component = w_e[0][0]
# last_component = w_e[0][299]
# entityType = w_e[0][300]
# print(type(first_component), first_component)
# print(type(last_component), last_component)
# print(type(entityType), entityType)

In [47]:
#  def createWordVectors_Entity(strategy, W):
#     '''
#     Create a N x (D+1) matrix where N is the total amount of words and 
#     D is the dimensionality of each word vector. 
#     The values of the last column are the entity type (PER - LOC - ORG - MISC - O).
    
#     strategy: 'concat', 'mean', 'frac_decay', 'exp_decay'.
#     W: window size
    
#     @return wordVectors_Entity matrix
#     '''
#     w2v_model = KeyedVectors.load('./models/word2vecGoogle.model')
#     D = w2v_model.vector_size
#     wordVectors_Entity = np.empty((0, D))
#     count = 0 # Used in the 'progress bar'
    
#     doc_filenames = os.listdir('./corpus_WiNER/docs_df/')
#     coarseNE_filenames = os.listdir('./corpus_WiNER/coarseNE_df/')
#     doc_filenames.sort()
#     coarseNE_filenames.sort()
    
#     for doc_f, coarseNE_f in zip(doc_filenames, coarseNE_filenames):
#         doc_df = pd.read_pickle('./corpus_WiNER/docs_df/'+ doc_f)
#         coarseNE_df = pd.read_pickle('./corpus_WiNER/coarseNE_df/'+ coarseNE_f)
#         art_IDs = coarseNE_df.art_ID.unique()
#         # We consider only the articles with at least one entity.
#         # That's why we iterate over the coarseNE's articles.
#         for art_ID in np.nditer(art_IDs):            
#             article_df = doc_df[doc_df.art_ID == art_ID]
#             m = wordVectorsFromArticle(article_df, strategy, D, W, w2v_model)
#             m = pd.DataFrame(m[0])
            
#             art_entities = coarseNE_df[coarseNE_df.art_ID == art_ID]
#             entity_list = entityListFromArticle(article_df, art_entities)            
#             m['entityType'] = entity_list
            
#             wordVectors_Entity = np.append(wordVectors_Entity, m.values)
            
#             if count % 10 == 0:
#                 print(count, end=' ')
#             count += 1
    
#     return wordVectors_Entity

In [59]:
def createWordVectors_Entity(strategy, W):
    '''  TODO: change description!
    Create a N x (D+1) matrix where N is the total amount of words and 
    D is the dimensionality of each word vector. 
    The values of the last column are the entity type (PER - LOC - ORG - MISC - O).
    
    strategy: 'concat', 'mean', 'frac_decay', 'exp_decay'.
    W: window size
    
    @return wordVectors_Entity matrix
    '''
    w2v_model = KeyedVectors.load('./models/word2vecGoogle.model')
    D = w2v_model.vector_size
    count = 0 # Used in the 'progress bar'
    
    doc_filenames = os.listdir('./corpus_WiNER/docs_df/')
    coarseNE_filenames = os.listdir('./corpus_WiNER/coarseNE_df/')
    doc_filenames.sort()
    coarseNE_filenames.sort()
    
    for doc_f, coarseNE_f in zip(doc_filenames, coarseNE_filenames):
#         wordVectors = np.empty((0, D))
#         entityVector = np.empty((0, 1))
        wordVectors = []
        entityVector = []
    
        doc_df = pd.read_pickle('./corpus_WiNER/docs_df/'+ doc_f)
        coarseNE_df = pd.read_pickle('./corpus_WiNER/coarseNE_df/'+ coarseNE_f)
        art_IDs = coarseNE_df.art_ID.unique()
        # We consider only the articles with at least one entity.
        # That's why we iterate over the coarseNE's articles.
        for art_ID in np.nditer(art_IDs):
                        
            article_df = doc_df[doc_df.art_ID == art_ID]
            art_entities = coarseNE_df[coarseNE_df.art_ID == art_ID]
            
#             start = time.time()
            w_vectors,_,_ = wordVectorsFromArticle(article_df, strategy, D, W, w2v_model)
#             end = time.time()
#             print('time 2: ', end-start)

#             filename = './corpus_WiNER/word_vectors/wv_'+ doc_f + '_art_' + str(int(art_ID))
#             np.savez_compressed(filename, *w_vectors)         
            
#             start = time.time()
            entity_list = entityListFromArticle(article_df, art_entities)            
#             end = time.time()
#             print('time 4: ', end-start)            
 
#             filename = './corpus_WiNER/entity_vectors/wv_'+ doc_f + '_art_' + str(int(art_ID))
#             np.savez_compressed(filename, entity_list)
    
    
    
            wordVectors += w_vectors
            
            entityVector += entity_list
            
            if count % 10 == 0:
                print(count, end=' ')
            count += 1

        np.savez_compressed('./corpus_WiNER/word_vectors/wv_'+doc_f, *wordVectors)
        # Probamos sin el * => guardar en un array de arrays 
#         np.savez_compressed('./corpus_WiNER/word_vectors/wv_'+doc_f, wordVectors)
        np.savez_compressed('./corpus_WiNER/entity_vectors/ev_'+doc_f, entityVector)
        break
    print('End process')

In [60]:
start = time.time()
createWordVectors_Entity(strategy='exp_decay', W=5)
end = time.time()
print('Demora: {}'.format(end - start))

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 

KeyboardInterrupt: 

In [ ]:
word_vectors = np.load('./corpus_WiNER/word_vectors/wv_doc_0.npz')

In [ ]:
# items2 = word_vectors.items()

In [ ]:
# for a_name, arr in word_vectors.iteritems():
#     print(a_name, len(arr))

In [ ]:
# len(df.loc[0,1])

Cantidad de oraciones: 54607542
Cantidad de artículos: 3239540